## Improving the baseline
This notebooks is for extracting the non seperate train file from the CUAD dataset

In [20]:
import json
# load json file
with open('../data/test_full.json') as json_file:
    data_test = json.load(json_file)

with open('../data/CUAD_v1/CUAD_v1.json') as json_file:
    data = json.load(json_file)

In [21]:
# Extracting the contracts in the test set to remove from the train
test_contracts = set()
q_set = set()
for contract in data_test['data']:
    for paragraph in contract['paragraphs']:
        q_set.add(len(paragraph['qas']))
        
    test_contracts.add(contract['title'])

In [22]:
_data = []
q_idx_dict = {}
contracts = []
discard_contracts = set()
for contract in data['data']:
    if contract['title'] in test_contracts:
        discard_contracts.add(contract['title'])
        continue
    contracts.append(contract)

            

In [23]:
print('Number of contracts in the test set:', len(test_contracts))
print('Number of contracts in the train set:', len(contracts))
print("Total number of contracts", len(data['data']))

Number of contracts in the test set: 102
Number of contracts in the train set: 408
Total number of contracts 510


In [24]:
data['data'] = contracts

In [25]:
# Validate the data
for contract in data['data']:
    if contract['title'] in discard_contracts:
        print("Issues found")

In [ ]:
# dump dataset
with open('../data/train_full.json', 'w') as outfile:
    json.dump(data, outfile)


## How many datapoints in the different dataset versions

In [28]:
import torch
import pandas as pd 
import numpy as np
import plotly.express as px


In [29]:
# Original dataset features
features_org = torch.load('../cuad_training/cuad_lightning/out/dataset-name_CUAD_model-type_roberta_only-first-ans_True_doc-stride_256_dataset-type_eval_predict-file-version_test_features')
# New dataset features
features_org = torch.load('../cuad_training/cuad_lightning/out/dataset-name_CUAD_model-type_roberta_only-first-ans_False_doc-stride_256_dataset-type_eval_predict-file-version_test_features')

In [57]:
from transformers.data.processors.squad import SquadFeatures
from collections import defaultdict

data = []
for feature in features_org:
    contract,question = feature.qas_id.split('__')
    
    data.append([contract, question, feature.is_impossible])

In [58]:
df = pd.DataFrame(data, columns=['contract', 'question','is_impossible'])

In [61]:
df_g = df.groupby(['question','is_impossible']).count()
df_g = df_g.reset_index()
df_g

,question,is_impossible,contract
0,Document Name,False,1
1,Document Name,True,5
2,Exclusivity,False,1
3,Exclusivity,True,6
4,License Grant,False,2
5,License Grant,True,5
6,Non-Transferable License,True,7
7,Unlimited/All-You-Can-Eat-License,False,2
8,Unlimited/All-You-Can-Eat-License,True,5


In [62]:
px.bar(df_g, x='question', y='contract', color='is_impossible', title='Number of contracts per question')